## UCL COMP0029 Individual Project for Year 3 BSc
### Robust Robotic Grasping Utilising Touch Sensing - Baseline Approach Notebook
This notebook contains the essential code for training and testing a supervised baseline approach to grasping. Given some tactile data, end effector poses, etc., it determines whether these constraints will produce a successful/unsuccessful grasp.

### 1. Load packages

In [60]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

Set device for `PyTorch` training

In [61]:
# Use GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### 2. Load datasets

Two datasets have been prepared from running the "collect sensory data" in the Pybullet simulation. These datasets are:
- `poses_ds.npy` that stores random 6d end effector poses generated by applying gaussian noise a manually selected base pose
- `training_ds.npy` that stores the tactile reading data collected from the DIGIT tactile sensors in the Pybullet simulation

The collected datasets are stored in `np.ndarray`s in a `.npy` file.

In [62]:
poses_ds_file_path = "poses_ds.npy"
poses_ds = np.load(poses_ds_file_path)
print(f"Shape of poses_ds: {poses_ds.shape}")

tactile_ds_file_path = "training_ds.npy"
tactile_ds = np.load(tactile_ds_file_path)
print(f"Shape of training_ds: {tactile_ds.shape}")

grasp_outcomes_ds_file_path = "grasp_outcomes.npy"
grasp_outcomes_ds = np.load(grasp_outcomes_ds_file_path)
print(f"Shape of grasp_outcomes_ds: {grasp_outcomes_ds.shape}")

Shape of poses_ds: (200, 6)
Shape of training_ds: (200, 240, 320, 3)
Shape of grasp_outcomes_ds: (200,)


In [63]:
### Check the number of successes and failures in the dataset
success = 0
failure = 0

for row in grasp_outcomes_ds:
    if row == 1:
        success += 1
    elif row == 0:
        failure += 1

print(f"Successes: {success} | Failures: {failure}")

Successes: 139 | Failures: 61


### 3. Approach 1 - Flattening images
We can try to flatten the tactile data into a 1D array instead.

In [64]:
flattened_tactile_data = [img.flatten() for img in tactile_ds]
flattened_tactile_data[0].shape

(230400,)

In [66]:
# Concatenate flattened_tactile_data with poses_ds
combined_data = np.concatenate((np.asarray(flattened_tactile_data), poses_ds), axis=1)
combined_data.shape

(200, 230406)

Now, we train this new datast on a logistic regression model.

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(combined_data, grasp_outcomes_ds, test_size=0.25, random_state=0)

simple_lr_model = LogisticRegression(random_state=0).fit(X_train, y_train)
simple_lr_model_predictions = simple_lr_model.predict(X_test)

print(f"Approach 1 Predictions:\n {simple_lr_model_predictions}")
print(f"Approach 1 Accuracy: {np.mean(y_test == simple_lr_model_predictions)*100:.2f}%")


Approach 1 Predictions:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Approach 1 Accuracy: 74.00%
